## Merging

In [ ]:
import os
import pandas as pd
import random
import gc  # for garbage collection

master_folder = r"D:\Stelco\Work\Dynamic Correlation\Key\Master Whole Phase"
phase_folder = r"D:\Stelco\Work\Dynamic Correlation\All coils remaining columns\Whole Phase"
output_folder = r"D:\Stelco\Work\Dynamic Correlation\Key\Merged whole Phase"

os.makedirs(output_folder, exist_ok=True)

file_names = os.listdir(master_folder)
csv_files = [f for f in file_names if f.endswith('.csv')]

for filename in csv_files:
    master_path = os.path.join(master_folder, filename)
    phase_path = os.path.join(phase_folder, filename)
    
    if not os.path.exists(phase_path):
        print(f"Warning: {phase_path} does not exist. Skipping.")
        continue
        
    df_master = pd.read_csv(master_path)
    df_phase = pd.read_csv(phase_path)
    
    df_merged = pd.merge(df_master, df_phase, on='Coil ID', how='left')
    
    output_path = os.path.join(output_folder, filename)
    df_merged.to_csv(output_path, index=False)

## Merging in Chunks

In [10]:
import os
import pandas as pd
import gc

master_folder = r"D:\Stelco\Work\Dynamic Correlation\Key\Master Whole Phase"
phase_folder = r"D:\Stelco\Work\Dynamic Correlation\All coils remaining columns\Whole Phase"
output_folder = r"F:\Merged Whole Phase"

os.makedirs(output_folder, exist_ok=True)
file_names = os.listdir(master_folder)
csv_files = [f for f in file_names if f.endswith('.csv')]

chunk_size = 50000  # Adjust based on your memory constraints

merge_keys = ['Coil ID', 'Time', 'Master Ramp', 'Coil Length [30ms]']

# Columns to keep from master file
master_columns_keep = [
    'Coil ID', 'Time', 'Master Ramp', 'Coil Length [30ms]','X4 Gauge Deviation', 'Stand 1 Gap Operator Offset', 'Stand 2 Gap Operator Offset',
    'Stand 3 Gap Operator Offset', 'Stand 4 Gap Operator Offset', 'S1 Operating Bending Trim', 'S2 Operating Bending Trim',
    'S3 Operating Bending Trim', 'S4 Operating Bending Trim',
    'X0 Gauge Deviation', 'X1 Gauge Deviation',
    'Stand 1-2 Tension Reference', 'Stand 2-3 Tension Reference', 'Stand 3-4 Tension Reference', 'Stand 1-2 Total Tension Feedback',
    'Stand 2-3 Total Tension Feedback', 'Stand 3-4 Total Tension Feedback', 'Exit Tension Reel Tension Reference',
    'Tension Reel Calculated Tension', 'Stand 1 Gap Bite Offset', 'Stand 1 Gap Stick Offset', 'Stand 1 Gap Thread Offset',
    'Stand 1 Thread Gap Setpoint', 'Stand 1 Run Gap Setpoint', 'Stand 2 Gap Bite Offset', 'Stand 2 Gap Stick Offset',
    'Stand 2 Gap Thread Offset', 'Stand 2 Thread Gap Setpoint', 'Stand 2 Run Gap Setpoint', 'Stand 3 Gap Bite Offset',
    'Stand 3 Gap Stick Offset', 'Stand 3 Gap Thread Offset', 'Stand 3 Thread Gap Setpoint', 'Stand 3 Run Gap Setpoint',
    'Stand 4 Gap Bite Offset', 'Stand 4 Gap Stick Offset', 'Stand 4 Gap Thread Offset', 'Stand 4 Thread Gap Setpoint',
    'Stand 4 Run Gap Setpoint', 'Stand 1 Predicted Run Force', 'Stand 2 Predicted Run Force', 'Stand 3 Predicted Run Force',
    'Stand 4 Predicted Run Force', 'Stand 1 - Total Force', 'Stand 2 - Total Force', 'Stand 2 - Total Force', 'Stand 2 - Total Force',
    'Stand 1 Top Current Feedback', 'Stand 2 Top Current Feedback', 'Stand 3 Top Current Feedback', 'Stand 4 Top Current Feedback',
    'Stand 1 Bottom Current Feedback', 'Stand 2 Bottom Current Feedback', 'Stand 3 Bottom Current Feedback', 'Stand 4 Bottom Current Feedback',
    'AGC GE Feedforward Hardness Number', 'AGC Alex Dynamic Feedforward Hardness Number', 'Stand 1 Total Bending Feedback',
    'Stand 2 Total Bending Feedback', 'Stand 3 Total Bending Feedback', 'Stand 4 OS Total Bending Feedback', 'Stand 4 DS Total Bending Feedback', 'Stand 1 - Operator Side Force',
    'Stand 1 - Drive Side Force', 'Stand 2 - Operator Side Force', 'Stand 2 - Drive Side Force', 'Stand 3 - Operator Side Force',  'Stand 3 - Drive Side Force',
    'Stand 4 - Operator Side Force', 'Stand 4 - Drive Side Force', 'Stand 1-2 Total Tension Feedback', 'Stand 2-3 Total Tension Feedback', 'Stand 3-4 Total Tension Feedback',
    'Stand 1 Drive Speed Feedback', 'Stand 2 Drive Speed Feedback', 'Stand 3 Drive Speed Feedback', 'Stand 4 Drive Speed Feedback',
    'Neet Oil Concentration', 'Stand 1-3 Solution Temperature', 'Stand 1-3 Solution System Pressure',
    'Stand 4 Solution System Pressure', 'Stand 1 Gap Eccentricity Trim', 'Stand 2 Gap Eccentricity Trim',
    'Stand 3 Gap Eccentricity Trim', 'Stand 4 Gap Eccentricity Trim', 'Morgoil OperBottom Bearing Outflow Temp Stand1',
    'Morgoil OperTop Bearing Outflow Temp Stand1', 'Morgoil DriveBottom Bearing Outflow Temp Stand1',
    'Morgoil DriveTop Bearing Outflow Temp Stand1', 'Morgoil OperBottom Bearing Outflow Temp Stand2',
    'Morgoil OperTop Bearing Outflow Temp Stand2', 'Morgoil DriveBottom Bearing Outflow Temp Stand2',
    'Morgoil DriveTop Bearing Outflow Temp Stand2', 'Morgoil OperBottom Bearing Outflow Temp Stand3',
    'Morgoil OperTop Bearing Outflow Temp Stand3', 'Morgoil DriveBottom Bearing Outflow Temp Stand3',
    'Morgoil DriveTop Bearing Outflow Temp Stand3', 'Morgoil OperBottom Bearing Outflow Temp Stand4',
    'Morgoil OperTop Bearing Outflow Temp Stand4', 'Morgoil DriveBottom Bearing Outflow Temp Stand4',
    'Morgoil DriveTop Bearing Outflow Temp Stand4'
]

total_files = len(csv_files)

for file_index, filename in enumerate(csv_files, start=1):
    print(f"\nProcessing file {file_index} of {total_files}: {filename}")

    master_path = os.path.join(master_folder, filename)
    phase_path = os.path.join(phase_folder, filename)

    if not os.path.exists(phase_path):
        print(f"  Warning: {phase_path} does not exist. Skipping.")
        continue

    print("  Reading phase file...")
    df_phase = pd.read_csv(phase_path)
    print(f"  Phase file loaded. Shape: {df_phase.shape}")

    output_path = os.path.join(output_folder, filename)
    first_chunk = True
    chunk_number = 1
    total_rows_written = 0

    for master_chunk in pd.read_csv(master_path, chunksize=chunk_size):
        print(f"    Processing chunk {chunk_number}, shape before selection: {master_chunk.shape}")

        # Keep only required columns, plus the merge keys if not already included
        # To ensure merge keys are present, add them to the list if missing
        for key in merge_keys:
            if key not in master_columns_keep:
                master_columns_keep.append(key)
        master_chunk = master_chunk[master_columns_keep]

        print(f"    Shape after keeping specified columns: {master_chunk.shape}")

        # Merge on the multiple keys
        df_merged = pd.merge(master_chunk, df_phase, on=merge_keys, how='left')
        print(f"    Merged chunk shape: {df_merged.shape}")

        df_merged.to_csv(output_path, mode='w' if first_chunk else 'a', header=first_chunk, index=False)
        total_rows_written += df_merged.shape[0]

        print(f"    Written {df_merged.shape[0]} merged rows to disk. Cumulative total: {total_rows_written}")

        first_chunk = False
        chunk_number += 1

        del master_chunk, df_merged
        gc.collect()

    del df_phase
    gc.collect()

    print(f"Finished processing file {file_index} of {total_files}: {filename}, total rows written: {total_rows_written}")

print("All files processed!")


Processing file 1 of 2988: 5386400.csv
  Reading phase file...
  Phase file loaded. Shape: (7458, 138)
    Processing chunk 1, shape before selection: (7458, 731)
    Shape after keeping specified columns: (7458, 105)
    Merged chunk shape: (7458, 239)
    Written 7458 merged rows to disk. Cumulative total: 7458
Finished processing file 1 of 2988: 5386400.csv, total rows written: 7458

Processing file 2 of 2988: 5386566.csv
  Reading phase file...
  Phase file loaded. Shape: (4136, 138)
    Processing chunk 1, shape before selection: (4136, 731)
    Shape after keeping specified columns: (4136, 105)
    Merged chunk shape: (4136, 239)
    Written 4136 merged rows to disk. Cumulative total: 4136
Finished processing file 2 of 2988: 5386566.csv, total rows written: 4136

Processing file 3 of 2988: 5422529.csv
  Reading phase file...
  Phase file loaded. Shape: (6668, 138)
    Processing chunk 1, shape before selection: (6668, 731)
    Shape after keeping specified columns: (6668, 105)
 

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3340\3769376445.py:65: DtypeWarning: Columns (20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_phase = pd.read_csv(phase_path)


  Phase file loaded. Shape: (5529, 138)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3340\3769376445.py:73: DtypeWarning: Columns (8,9,12,14,33,35,42,46,47,49,50,51,53,55,66,70,75,79,80,84,85,87,93,104,116,125,136,139,142) have mixed types. Specify dtype option on import or set low_memory=False.
  for master_chunk in pd.read_csv(master_path, chunksize=chunk_size):


    Processing chunk 1, shape before selection: (5529, 731)
    Shape after keeping specified columns: (5529, 105)
    Merged chunk shape: (5529, 239)
    Written 5529 merged rows to disk. Cumulative total: 5529
Finished processing file 968 of 2988: 5455312.csv, total rows written: 5529

Processing file 969 of 2988: 5455315.csv
  Reading phase file...
  Phase file loaded. Shape: (4006, 138)
    Processing chunk 1, shape before selection: (4006, 731)
    Shape after keeping specified columns: (4006, 105)
    Merged chunk shape: (4006, 239)
    Written 4006 merged rows to disk. Cumulative total: 4006
Finished processing file 969 of 2988: 5455315.csv, total rows written: 4006

Processing file 970 of 2988: 5455318.csv
  Reading phase file...
  Phase file loaded. Shape: (2666, 138)
    Processing chunk 1, shape before selection: (2666, 731)
    Shape after keeping specified columns: (2666, 105)
    Merged chunk shape: (2666, 239)
    Written 2666 merged rows to disk. Cumulative total: 2666


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3340\3769376445.py:65: DtypeWarning: Columns (20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_phase = pd.read_csv(phase_path)


  Phase file loaded. Shape: (9689, 138)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3340\3769376445.py:73: DtypeWarning: Columns (8,9,10,12,14,28,29,31,33,35,42,46,47,49,50,51,53,55,61,66,67,70,74,75,79,80,83,84,85,87,93,96,104,106,116,118,119,123,124,125,126,127,129,135,136,139,142) have mixed types. Specify dtype option on import or set low_memory=False.
  for master_chunk in pd.read_csv(master_path, chunksize=chunk_size):


    Processing chunk 1, shape before selection: (9689, 731)
    Shape after keeping specified columns: (9689, 105)
    Merged chunk shape: (9689, 239)
    Written 9689 merged rows to disk. Cumulative total: 9689
Finished processing file 2658 of 2988: 5505604.csv, total rows written: 9689

Processing file 2659 of 2988: 5505609.csv
  Reading phase file...
  Phase file loaded. Shape: (3524, 138)
    Processing chunk 1, shape before selection: (3524, 731)
    Shape after keeping specified columns: (3524, 105)
    Merged chunk shape: (3524, 239)
    Written 3524 merged rows to disk. Cumulative total: 3524
Finished processing file 2659 of 2988: 5505609.csv, total rows written: 3524

Processing file 2660 of 2988: 5505613.csv
  Reading phase file...
  Phase file loaded. Shape: (3529, 138)
    Processing chunk 1, shape before selection: (3529, 731)
    Shape after keeping specified columns: (3529, 105)
    Merged chunk shape: (3529, 239)
    Written 3529 merged rows to disk. Cumulative total: 3

## Creating Sample of 50

In [1]:
import os
import shutil
import random
import zipfile

# Input folders
master_folder = r"D:\Stelco\Work\Dynamic Correlation\Key\Master whole Phase"       # Old IBA
phase_folder = r"D:\Stelco\Work\Dynamic Correlation\All coils remaining columns\Whole Phase"  # New IBA

# Output ZIP file paths
master_zip_path = r"D:\Stelco\Work\Dynamic Correlation\Old_IBA_50.zip"
phase_zip_path = r"D:\Stelco\Work\Dynamic Correlation\New_IBA_50.zip"

# Temp folders
temp_master = r"D:\Stelco\Work\Dynamic Correlation\Temp_Master"
temp_phase = r"D:\Stelco\Work\Dynamic Correlation\Temp_Phase"

# Create temp folders
os.makedirs(temp_master, exist_ok=True)
os.makedirs(temp_phase, exist_ok=True)

# Get matching CSV files
master_files = set(f for f in os.listdir(master_folder) if f.endswith('.csv'))
phase_files = set(f for f in os.listdir(phase_folder) if f.endswith('.csv'))
common_files = list(master_files & phase_files)

# Sample 50 filenames
sample_files = random.sample(common_files, 50)

# Copy sampled files to respective temp folders
for filename in sample_files:
    shutil.copy(os.path.join(master_folder, filename), os.path.join(temp_master, filename))
    shutil.copy(os.path.join(phase_folder, filename), os.path.join(temp_phase, filename))